In [ ]:
# 필요한 패키지 설치
!pip install -q -U immutabledict sentencepiece

# Gemma PyTorch 라이브러리 클론 및 파일 이동
!git clone https://github.com/google/gemma_pytorch.git
!mkdir -p /kaggle/working/gemma/
!mv gemma_pytorch/gemma/* /kaggle/working/gemma/

In [ ]:
# 필수 라이브러리 임포트
import sys
import os
import torch
import contextlib
sys.path.append("/kaggle/working/gemma/") 

# Gemma 관련 모듈 임포트
from gemma.config import GemmaConfig, get_model_config
from gemma.model import GemmaForCausalLM

# 모델을 불러오기 위한 경로 설정
VARIANT = "2b-v2"  # 모델 버전
MACHINE_TYPE = "cuda" if torch.cuda.is_available() else "cpu"  # 디바이스 선택
weights_dir = '/kaggle/input/gemma-2/pytorch/gemma-2-2b-it/1'  # 모델 체크포인트 경로
weights_file = os.path.join(weights_dir, "model.ckpt")  # 가중치 파일 경로
tokenizer_path = os.path.join(weights_dir, "tokenizer.model")  # 토크나이저 경로

# 텐서 타입 설정을 위한 context manager 정의
@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
    """Sets the default torch dtype to the given dtype."""
    torch.set_default_dtype(dtype)
    yield
    torch.set_default_dtype(torch.float)

# 모델 설정 및 로드
model_config = get_model_config(VARIANT)
model_config.tokenizer = tokenizer_path  # 토크나이저 경로 설정
device = torch.device(MACHINE_TYPE)  # 디바이스 설정

with _set_default_tensor_type(model_config.get_dtype()):
    model = GemmaForCausalLM(model_config)  # 모델 초기화
    model.load_weights(weights_file)  # 가중치 로드
    model = model.to(device).eval()  # 모델 평가 모드로 전환


In [ ]:
# Let's inspect the provided Python file to understand its structure and content.
file_path = '/kaggle/input/llm-20-questions/keywords.py'

# Read the content of the provided file
with open(file_path, 'r') as file:
    keywords_content = file.read()

keywords_content

In [ ]:
 import textwrap
from IPython.display import display, Markdown
import torch
import contextlib
import os
from gemma.config import get_model_config
from gemma.model import GemmaForCausalLM

# 모델 로드 함수
def load_model(VARIANT, device, model_path):
    WEIGHTS_PATH = model_path

    @contextlib.contextmanager
    def _set_default_tensor_type(dtype: torch.dtype):
        """Sets the default torch dtype to the given dtype."""
        torch.set_default_dtype(dtype)
        yield
        torch.set_default_dtype(torch.float)

    # 모델 설정
    model_config = get_model_config(VARIANT)
    model_config.tokenizer = os.path.join(WEIGHTS_PATH, "tokenizer.model")  # 토크나이저 경로
    model_config.quant = "quant" in VARIANT

    # 모델 로드
    with _set_default_tensor_type(model_config.get_dtype()):
        model = GemmaForCausalLM(model_config)
        ckpt_path = os.path.join(WEIGHTS_PATH, 'model.ckpt')  # 체크포인트 경로
        model.load_weights(ckpt_path)
        model = model.to(device).eval()

    return model

# ChatState 클래스 (대화 관리)
class ChatState():
    """
    Manages the conversation history for the "20 Questions" game chatbot.
    """
    __START_TURN_USER__ = "<start_of_turn>user\n"
    __START_TURN_MODEL__ = "<start_of_turn>model\n"
    __END_TURN__ = "<end_of_turn>\n"

    def __init__(self, model, device, system=""):
        """
        Initializes the chat state.
        Args:
            model: The language model to use for generating responses.
            device: The device (e.g., "cuda" or "cpu") to use for inference.
            system: (Optional) System instructions or bot description.
        """
        self.model = model
        self.device = device
        self.system = system
        self.history = []
        self.question_count = 0  # To track how many questions the bot has asked

    def add_to_history_as_user(self, message):
        """Adds a user message to the history with start/end turn markers."""
        self.history.append(self.__START_TURN_USER__ + message + self.__END_TURN__)

    def add_to_history_as_model(self, message):
        """Adds a model response to the history with start/end turn markers."""
        self.history.append(self.__START_TURN_MODEL__ + message + self.__END_TURN__)

    def get_history(self):
        """Returns the entire chat history as a single string."""
        return "".join(self.history)

    def get_full_prompt(self):
        """Builds the prompt for the language model, including history and system description."""
        prompt = self.get_history() + self.__START_TURN_MODEL__
        if len(self.system) > 0:
            prompt = self.system + "\n" + prompt
        return prompt

    def send_message(self, message):
        """Handles sending a user message and getting a model response."""
        self.add_to_history_as_user(message)
        prompt = self.get_full_prompt()

        # Generate the model's response
        response = self.model.generate(prompt, device=self.device)

        # Handle the response format (if list or string)
        if isinstance(response, list):
            result = response[0]
        else:
            result = response.replace(prompt, "")  # Remove prompt from the response

        self.add_to_history_as_model(result)
        self.question_count += 1

        return result

# 시스템 프롬프트 설정 (20 Questions 게임을 위한 시스템 설명)
system_prompt = (
    "You are an AI assistant playing the 20 Questions game. "
    "In this game, you should ask the most relevant and logical yes-or-no questions. "
    "The goal is to identify a specific object, place, or thing that the user is thinking of. "
    "Ask questions that will help you narrow down the possibilities."
)

# 모델 초기화 (사용 중인 모델 버전에 맞게 수정)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
weights_dir = '/kaggle/input/gemma-2/pytorch/gemma-2-2b-it/1'  # 모델 파일 경로
gemma_lm = load_model("2b-v2", device, weights_dir)

# 챗봇 상태 초기화 (시스템 설명 포함)
chat = ChatState(gemma_lm, device, system=system_prompt)

# 대화 출력 함수
def display_chat(prompt, text):
    formatted_prompt = "🙋‍♂️ " + prompt + "\n"
    text = textwrap.indent(text, '> ')
    formatted_text = "🤖\n" + text + "\n"
    display(Markdown(formatted_prompt + formatted_text))

# 20 Questions 게임 함수 수정
def play_20_questions(chat):
    print("Let's play 20 Questions!")
    
    # 첫 질문을 모델이 생성
    question = chat.send_message("Let's play 20 Questions. I'll start by asking: Is it a living thing?")
    display_chat("Let's play 20 Questions. I'll start by asking: Is it a living thing?", question)
    
    # 20번의 질문을 진행 (또는 모델이 정답을 맞출 때까지)
    for _ in range(19):  # 최대 20번 질문
        answer = input("Your answer (yes/no): ").strip().lower()  # 유저의 대답 받기
        
        # 'yes' 또는 'no'에 따라 다음 질문을 생성
        if answer == "yes" or answer == "no":
            next_question = chat.send_message(answer)
            display_chat(answer, next_question)
        else:
            print("Please answer with 'yes' or 'no'.")
            continue
        
        # 모델이 구체적인 물체나 장소를 추측하려는 경우 ('Is it a...' 또는 'Is it the...' 형태로 추측)
        if next_question.lower().startswith("is it a") or next_question.lower().startswith("is it the"):
            # 모델이 물체나 장소를 추측하려 할 때
            print("Bot is trying to guess...")
            print("The model's guess: ", next_question)
            
            # 정답 확인 (특정 물체나 장소를 추측했을 때만 종료)
            final_answer = input("Is the model's guess correct? (yes/no): ").strip().lower()
            if final_answer == "yes":
                print("The model guessed correctly! The game is over.")
                break
            else:
                print("The model guessed wrong. Continuing the game...")

        # 모델이 추측을 시도하는데 명확한 답이 아닌 경우에는 계속 진행
        else:
            print("The model is still narrowing down the options...")
    
    print("End of game!")

# 게임 시작
play_20_questions(chat)


**It's done!!! check it out!**